In [1]:
# Importações do seu pacote
from tiny_diffusion_model import TimeInputMLP, ScheduleLogLinear, training_loop, samples
from tiny_diffusion_model import BarnsleyFern, plot_batch, moving_average

import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

# Configurações
BATCH_SIZE = 256
EPOCHS = 200
LR = 1e-3
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# 1. Dataset
dataset = BarnsleyFern(10000)
loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# 2. Modelo e scheduler
model = TimeInputMLP(dim=2).to(DEVICE)
schedule = ScheduleLogLinear(N=100)

# 3. Treinamento
losses = []
for state in training_loop(loader, model, schedule, epochs=EPOCHS, lr=LR):
    losses.append(state.loss.item())
    if state._ % 10 == 0:
        print(f"Epoch {state._}: Loss = {state.loss.item():.4f}")

# 4. Visualização da perda
plt.plot(moving_average(losses, 10))
plt.title("Training Loss (Moving Average)")
plt.xlabel("Step")
plt.ylabel("Loss")
plt.grid()
plt.show()

# 5. Geração de amostras
sigmas = schedule.sample_sigmas(steps=50).to(DEVICE)

samples_list = []
for sample in samples(model, sigmas, batchsize=1024):
    samples_list.append(sample.cpu())

# 6. Visualizar imagem final
final_sample = samples_list[-1].numpy()
plt.scatter(final_sample[:, 0], final_sample[:, 1], s=0.5, alpha=0.7)
plt.title("Generated Sample (Final Step)")
plt.axis('equal')
plt.grid(True)
plt.show()


ModuleNotFoundError: No module named 'tiny_diffusion_model'